# Resnet-50 Model Classification

In [ ]:
# Import the necessary libraries
# Import libraries
import time
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn.functional as F
from sklearn.metrics import confusion_matrix, classification_report


# hyper-parameters
EPOCHS = 5
BATCH_SIZE = 128
LEARNING_RATE = 1e-4

# Base Path
BASE_PATH = 'archive/'

### Set-up GPU acceleration

In [ ]:
device = 'mps' if torch.backends.mps.is_available() else "cpu"

### Import train/test datesets and use transform function

In [ ]:
# Transform data to Tensor object to load to Resnet model
data_transforms = {
    'train': transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()]),
    'test':  transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])
}


# Load the training and testing dataset 
image_datasets = {
    'train': torchvision.datasets.ImageFolder(BASE_PATH + 'Training', data_transforms['train']),
    'test':  torchvision.datasets.ImageFolder(BASE_PATH + 'Testing',  data_transforms['test'])
}

# Create the Dataloader object 
dataloaders = {
    'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=128,shuffle=True),
    'test' : torch.utils.data.DataLoader(image_datasets['test'],  batch_size=32, shuffle=False)  
}

### Use Resnet-50 Model

We use transfer learning in order to train the latest layer of a Resnet model in order to get a baseline of the performance for the state-of-art image classifier.

In [ ]:
# Load the model from transformers library
model = models.resnet50(pretrained=True)

# Define the number of features we are going to training in this project, which are the last layers parameters
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, len(image_datasets['train'].classes))
model = model.to(device)


# Define the loss function and the optimizer function
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

### Create the training and testing procedures

In [ ]:
# Define the train procedure for the model
def train(model, dataloader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    accuracy = correct / total
    return running_loss, accuracy


# Define the test/evaluation procedure for the model
def evaluate(model, dataloader):
    model.eval()
    predictions = []
    true_labels = []
    outputs = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            output = model(inputs)
            _, predicted = torch.max(output, 1)

            predictions.extend(predicted.tolist())
            true_labels.extend(labels.tolist())
            outputs.extend(output.tolist())

    return predictions, true_labels, outputs

#### Start with the training procedure

In [ ]:
file_training = open('train_resnet-50.txt', 'a')

start_time = time.time()

for epoch in range(EPOCHS):
    train_loss, train_accuracy = train(model, dataloaders['train'], criterion, optimizer)  
    print(f"Epoch: {epoch+1}/{EPOCHS}")
    print(f"Train Loss: {train_loss:.4f} - Train Accuracy: {train_accuracy:.4f}")
    file_training.write('Epoch: {}/{} '.format(epoch+1, EPOCHS) + 'Train Loss: {0} - Train Accuracy: {1} '.format(train_loss, train_accuracy) + '\n' )

finish_time = time.time()

file_training.write('Time needed to train for {0} epochs is: {1}'.format(EPOCHS, finish_time-start_time))
file_training.close()
torch.save(model.state_dict(), 'model-50.pth')

#### Testing procedure

In [ ]:
# Create the a custom Resnet-50 model that fits our problem of 4 classes
class CustomResnet(nn.Module):
    def __init__(self, num_classes):
        super(CustomResnet, self).__init__()
        self.resnet = models.resnet50(pretrained=False)
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_ftrs, num_classes)

    def load_backbone_weights(self, state_dict):
        self.resnet.load_state_dict(state_dict, strict=False)

    def forward(self, x):
        return self.resnet(x)

In [ ]:
# Create the test file 
new_model =  CustomResnet(len(image_datasets['train'].classes))
new_model.load_backbone_weights(torch.load('model-50.pth'))
new_model = new_model.to(device)
new_model.eval()
test_file = open('test_resnet-50.txt', 'a+')

for epoch in range(EPOCHS):
    test_predictions, test_true_labels, test_outputs = evaluate(new_model, dataloaders['test'])
    test_predictions = torch.tensor(test_predictions, dtype=torch.long).to(device)  # Convert to Long
    test_loss = criterion(torch.tensor(test_outputs).to(device), torch.tensor(test_true_labels).to(device))
    
    test_predictions_list = test_predictions.tolist()
    test_true_labels_list = test_true_labels

    test_accuracy = sum([1 for i, j in zip(test_predictions_list, test_true_labels_list) if i == j]) / len(test_predictions_list)
  
    print(f"Epoch: {epoch+1}/{EPOCHS}")
    print(f"Test Loss: {test_loss:.4f} - Test Accuracy: {test_accuracy:.4f}")
    test_file.write('Epoch: {}/{} '.format(epoch+1, EPOCHS) + 'Test Loss: {0} - Test Accuracy: {1} '.format(test_loss, test_accuracy) + '\n')

# Calculate confusion matrix
test_predictions, test_true_labels, outputs = evaluate(new_model, dataloaders['test'])
confusion_mtx = confusion_matrix(test_true_labels, test_predictions)
print("Confusion Matrix:")
print(confusion_mtx)
test_file.write('Confusion Matrix: \n{}\n'.format(confusion_matrix))

# Calculate classification report
classification_rep = classification_report(test_true_labels, test_predictions)
print("Classification Report:")
print(classification_rep)
test_file.write('Classification Matrix: \n{}'.format(classification_rep))
test_file.close()